In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time

from IPython import display

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [ ]:
BUFFER_SIZE = 60000
BASELINE_BATCH_SIZE = 256
NEW_BATCH_SIZE = 128
NOISE_DIM = 100
NEW_NOISE_DIM = 50
EPOCHS = [50]
BASELINE_LR = 1e-4
NEW_LR = 1e-3


In [ ]:
# Baseline model hyperparameters
BASELINE_PARAMS = {
    "noise_dim": NOISE_DIM,
    "batch_size": BASELINE_BATCH_SIZE,
    "learning_rate": BASELINE_LR
}

# New hyperparameters
NEW_PARAMS = {
    "noise_dim": NEW_NOISE_DIM,
    "batch_size": NEW_BATCH_SIZE,
    "learning_rate": NEW_LR
}

In [ ]:
def make_generator_model(noise_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(7 * 7 * 256, use_bias = False, input_shape = (noise_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:

# Define discriminator and generator
generator_baseline = make_generator_model(BASELINE_PARAMS["noise_dim"])
generator_new = make_generator_model(NEW_PARAMS["noise_dim"])
discriminator = make_discriminator_model()
# Define optimizers
generator_optimizer_baseline = tf.keras.optimizers.Adam(BASELINE_PARAMS["learning_rate"])
generator_optimizer_new = tf.keras.optimizers.Adam(NEW_PARAMS["learning_rate"])
discriminator_optimizer = tf.keras.optimizers.Adam(BASELINE_PARAMS["learning_rate"])  # Same learning rate for discriminator

c:\Users\sharh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\sharh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [ ]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

In [ ]:
# Function to calculate discriminator loss
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [ ]:
@tf.function
def train_step(images, generator, discriminator_optimizer, generator_optimizer):
    baseline_noise_dim = BASELINE_PARAMS["noise_dim"]
    new_noise_dim = NEW_PARAMS["noise_dim"]

    #Generate noise tensors using the noise dimensions provided
    baseline_noise= tf.random.normal([BASELINE_BATCH_SIZE, baseline_noise_dim])
    new_noise= tf.random.normal([NEW_BATCH_SIZE, new_noise_dim])

    #Select appropriate noise tensor
    noise = baseline_noise if generator == generator_baseline else new_noise

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      generated_images = generator(noise, training=True)
      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
tf.config.run_functions_eagerly(True)
train_step(train_images, generator_new, discriminator_optimizer, generator_optimizer_new)

In [ ]:
#Define a checkpoint
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer_baseline, discriminator_optimizer=discriminator_optimizer, generator=generator_baseline,
#                                  discriminator=discriminator)
checkpoint = tf.train.Checkpoint()

In [ ]:
def generate_and_save_images(generator, epoch, test_input, save_dir='images/'):
  print(save_dir)
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = generator(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  os.makedirs(save_dir, exist_ok=True)

  plt.savefig(os.path.join(save_dir, 'image_at_epoch_{:04d}.png'.format(epoch)))  # Save images in the specified directory
  plt.show()


In [ ]:
# Generate a random seed
seed_baseline = tf.random.normal([16, BASELINE_PARAMS["noise_dim"]])  # You can adjust the first dimension (16) as needed
seed_new = tf.random.normal([16, NEW_PARAMS["noise_dim"]])

In [ ]:
# Function to train the model
def train(dataset, epochs, generator, discriminator, generator_optimizer, discriminator_optimizer, seed):
    for num_epochs in epochs:
        for epoch in range(num_epochs):
            start = time.time()

            for image_batch in dataset:
                baseline_noise= tf.random.normal([BASELINE_BATCH_SIZE, BASELINE_PARAMS["noise_dim"]])
                new_noise= tf.random.normal([NEW_BATCH_SIZE, NEW_PARAMS["noise_dim"]])
                noise = baseline_noise if generator == generator_baseline else new_noise

                train_step(image_batch, generator, discriminator_optimizer, generator_optimizer)
                # Print generator and noise information
                # print("Generator:", generator)
                # print("Noise shape:", noise.shape)

            # Update and display progress
            display.clear_output(wait=True)
            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

            # Save the model every 15 epochs
            if (epoch + 1) % 15 == 0:
                checkpoint.save(file_prefix=checkpoint_prefix)
            # Save the model after 50 epochs
            if (epoch + 1) == 50:
                generate_and_save_images(generator, epoch + 1, seed)
             # Print generator and noise information
            print("Generator:", generator)
            print("Noise shape:", noise.shape)

  # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           sum(epochs),
                           seed)
  #Save for each set of epochs
    checkpoint.save(file_prefix=checkpoint_prefix)

#Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BASELINE_BATCH_SIZE)

#train with baseline hyperparameters
train(train_dataset, [50], generator_baseline, discriminator, generator_optimizer_baseline, discriminator_optimizer, seed_baseline)
#train with new hyperparameters
train(train_dataset, [50], generator_new, discriminator, generator_optimizer_new, discriminator_optimizer, seed_new)
